In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [3]:
feed.routes.route_type.iat[0] = 2  # Create another route type
trip_stats = feed.compute_trip_stats()

In [4]:
#dates = ["20100101"]
week = feed.get_first_week()
dates = [week[0], week[2]] + ["20100101"]
f = feed.compute_feed_stats(trip_stats, dates=dates, split_route_types=True)
f.T

,0,1,2,3
route_type,2,3,2,3
num_stops,66,386,66,386
num_routes,1,19,1,19
num_trips,59,563,59,563
num_trip_starts,59,563,59,563
num_trip_ends,58,559,58,559
service_distance,1894.22,11879.8,1894.22,11879.8
service_duration,57.35,415.25,57.35,415.25
service_speed,33.0292,28.6088,33.0292,28.6088
peak_num_trips,5,35,5,35


In [8]:
freq = "12H"
sts = feed.compute_stop_time_series(dates, freq=freq, split_directions=True)
rts = feed.compute_route_time_series(trip_stats, dates, freq=freq)
fts = feed.compute_feed_time_series(trip_stats, dates, freq=freq, split_route_types=True)

In [6]:
fts

indicator  num_trip_ends        num_trip_starts        num_trips         \
route_type             2      3               2      3         2      3   
datetime                                                                  
2014-05-26          58.0  559.0            59.0  563.0      59.0  563.0   
2014-05-27           NaN    NaN             NaN    NaN       NaN    NaN   
2014-05-28          58.0  559.0            59.0  563.0      59.0  563.0   

indicator  service_distance               service_duration          \
route_type                2             3                2       3   
datetime                                                             
2014-05-26      1894.223063  11879.804171            57.35  415.25   
2014-05-27              NaN           NaN              NaN     NaN   
2014-05-28      1894.223063  11879.804171            57.35  415.25   

indicator  service_speed           
route_type             2        3  
datetime                           
2014-05-26     33.029173  28.6088  
2014-05-27           NaN      NaN  
2014-05-28     33.029173  28.6088

In [158]:
# Slicing 
display(fts.loc[:, ("num_trips", slice(None))])
display(fts.xs("num_trips", axis="columns"))

indicator  num_trips       
route_type         2      3
datetime                   
2014-05-26      59.0  563.0
2014-05-27       NaN    NaN
2014-05-28      59.0  563.0

route_type,2,3
datetime,,
2014-05-26,59.0,563.0
2014-05-27,NaN,NaN
2014-05-28,59.0,563.0


In [67]:
f = fts = feed.compute_feed_time_series(trip_stats, dates[:1], freq="12H", split_route_types=False)
display(f)
g = gt.unstack_time_series(f)

# columns = [c for c in g.columns if c not in ["datetime", "value"]]
# h = g.pivot_table(index="datetime", columns=columns).value.sort_index(
#     axis="columns"
# )

# hours = (h.index[1] - h.index[0]).components.hours
# if hours != 0:
#     freqy = f"{hours}H"
# else:
#     freqy = "D"

# print(freqy)

h = gt.restack_time_series(g)
h

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-26 00:00:00,214,243,242,5020.407336,175.833333,28.552080
2014-05-26 12:00:00,403,379,408,8753.619898,296.766667,29.496641


indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-26 00:00:00,214.0,243.0,242.0,5020.407336,175.833333,28.552080
2014-05-26 12:00:00,403.0,379.0,408.0,8753.619898,296.766667,29.496641


In [68]:
f.index

DatetimeIndex(['2014-05-26 00:00:00', '2014-05-26 12:00:00'], dtype='datetime64[ns]', name='datetime', freq='12H')

In [66]:
delta.components.hours

NameError: name 'delta' is not defined

In [ ]:
feed1 = feed.copy()
cal = feed1.calendar.copy()
cal["monday"] = 0
feed1.calendar = cal
feed1.compute_feed_time_series(trip_stats, dates, freq="12H")

In [ ]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()